In [1]:
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_huggingface import HuggingFaceEndpointEmbeddings
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage, AIMessage, trim_messages, BaseMessage
from typing_extensions import TypedDict
from langchain_core.messages.utils import count_tokens_approximately

load_dotenv()


True

In [2]:
embeddings = HuggingFaceEndpointEmbeddings(
    model="sentence-transformers/all-MiniLM-L6-v2", task="feature-extraction"
)

In [3]:
class IndexState(TypedDict):
    pdf_path: str
    output_dir: str


In [4]:
def create_vector(state1: IndexState) -> FAISS:
    loader = PyPDFLoader(state1["pdf_path"])
    pdf_file = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks = text_splitter.split_documents(pdf_file)

    db = FAISS.from_documents(documents= chunks, embedding=embeddings)
    db.save_local(state1["output_dir"])
    return {}

In [ ]:
def call_model(state: MessagesState):
    query = state["messages"][-1].content
    db = FAISS.load_local(r"./vectordb", embeddings=embeddings,
                           allow_dangerous_deserialization=True)
    docs = db.similarity_search(query=query, k=4)
    docs_page_content = " ".join([d.page_content for d in docs])

    
    promt_template = ChatPromptTemplate.from_messages([
        ("system","You are an helpful assistant. Answer to the questions asked "
        "based on the given context. Question : {question}, context:{context}."
        "If the answer is not in the context just say I don't know!"),
        MessagesPlaceholder(variable_name="messages")
        ])
    
    prompt = promt_template.invoke({"question":query, 
                                    "context": docs_page_content,
                                    "messages":state["messages"]})
    
    llm = ChatOpenAI(
        base_url="https://router.huggingface.co/v1",
        model="meta-llama/Llama-3.2-3B-Instruct:together",
        temperature=0.2, 
    )
    
    # print("Current conversation history:")
    # for m in state["messages"]:
    #     print(type(m).__name__, ":", m.content)

    response = llm.invoke(prompt)
    return {"messages": [response]}

In [6]:
def trimmer(state: MessagesState):
    # msgs = state["messages"]
    # # Flatten: handle accidental nesting like [ [HumanMessage(...), AIMessage(...)] , ... ]
    # flat: list[BaseMessage] = []
    # for m in msgs:
    #     if isinstance(m, list):
    #         flat.extend(m)
    #     else:
    #         flat.append(m)

    trim = trim_messages(
        state["messages"],
        max_tokens = 60,
        strategy = "last",
        token_counter = count_tokens_approximately,
        include_system = True,
        allow_partial = False,
        start_on = "human",
        end_on = ("human", "tool"),

    )

    return {"messages": trim}

In [7]:
def index_pipeline():
    workflow1 = StateGraph(state_schema=IndexState)

    workflow1.add_node("vectorize", create_vector)
    workflow1.add_edge(START, "vectorize")
    workflow1.add_edge("vectorize", END)

    app1 = workflow1.compile()

    return app1

In [8]:
def chat_pipeline():
    workflow = StateGraph(state_schema=MessagesState)

    workflow.add_node("trimming", trimmer)
    workflow.add_node("model", call_model)

    workflow.add_edge(START, "trimming")
    workflow.add_edge("trimming", "model")
    workflow.add_edge("model", END)

    app = workflow.compile(checkpointer=MemorySaver())
    return app

In [ ]:
# indexpp = index_pipeline()
# vectordb = indexpp.invoke({"pdf_path":r".\vectordb", "output_dir":r".\Chatbot with rag"})

In [10]:
cp = chat_pipeline()

In [11]:
config = {"configurable": {"thread_id":"ragchat1"}}

In [12]:
# query = "The chatbot can perform rag and also store convo history"

# input_messages = [HumanMessage(query)]
# output = cp.invoke({"messages":input_messages}, config)
# output["messages"][-1].pretty_print()

In [ ]:
print("Type 'exit' to quit.\n")

while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        break
    
    input_messages = [HumanMessage(query)]
    
    # Stream response
    print("Bot:", end=" ")
    for chunk, meta in cp.stream({"messages": input_messages}, config, stream_mode="messages",):
        if isinstance(chunk, AIMessage):
            print(chunk.content, end="", flush=True)
    print()

Type 'exit' to quit.

Bot: Hi Karthik! How can I assist you today?
Bot: 